## Teste Data Science - Parte 1
### Raphael Ballet 
raphaelballet@gmail.com

---
### Objetivo: 
Criar um analisador de sentimentos de avaliações de filme obtidos pelo Rotten Tomatoes, que estão disponíveis no seguinte [*dataset*](http://www.cs.cornell.edu/people/pabo/movie-review-data/). O sistema deve classificar as avaliações entre "positivas" e "negativas", o que refletiria a opinião de cada avaliador sobre o filme.

---
### Solução:
#### Introdução
Esta seção apresenta uma breve introdução ao problema, demonstrando sua motivação e dificuldades. Como diversos outros problemas de *Natural Language Processing* ([NLP](https://en.wikipedia.org/wiki/Natural_language_processing)), o problema de extração de sentimentos a partir de textos tem grande importância para a evolução dos sistemas de inteligência artificial (IA) [[1]](https://www.brandwatch.com/blog/understanding-sentiment-analysis/). A capacidade de identificar automaticamente o sentimento de um texto permite aos sistemas de IA realizar tarefas que até então eram exclusivamente humanas. Aliás, mesmo os próprios humanos possuem dificuldade nessa tarefa, já que são somos unânimes em relação a esse tipo de classificação [[2]](http://mashable.com/2010/04/19/sentiment-analysis/#KrPG2rUv45qh). Diversos fatores como a presença de frases ambíguas, sarcasmo, frases negativas, expressões idiomáticas e contexto, influenciam a capacidade de seres humanos e máquinas em suas classificações [[3]](http://www.cs.cornell.edu/home/llee/omsa/omsa.pdf). Portanto, justamente pela complexidade do problema e seu grande potencial de aplicações é que diversos autores se motivaram a dedicar suas pesquisas a essa classe de problemas. 

#### Metodologia
Nesta seção, eu apresento detalhadamente as etapas que julgo necessárias para resolver o problema em questão.

- **Etapa 0**: O primeiro passo que tomei ao iniciar o problema foi criar um entendimento preliminar sobre a análise de sentimentos. Como não tenho experiência com o assunto, é importante entender as motivações e principais problemas que envolvem esse assunto. Basicamente, o que observei foi que os métodos mais comumente aplicados para a resolução desse problema se dividem nos métodos baseados em métodos estatísticos e *machine learning* [[4]](http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf) [[5]](http://www.cs.cornell.edu/home/llee/papers/cutsent.pdf) [[6]](http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=9AF55F153515B1D5C27B68B9460B02AE?doi=10.1.1.129.4132&rep=rep1&type=pdf) (veja figura abaixo retirada de [[7]](http://www.sciencedirect.com/science/article/pii/S2090447914000550)), os quais procuram encontrar relações de sentimento ou *labels* a partir das palavras contidas no texto que se deseja analisar em relação a outros exemplos (em sua maioria, são problemas supervisionados); e também os métodos baseados em estruturas léxicas e/ou semânticas do texto [[8]](https://arxiv.org/pdf/cs/0212032.pdf), que tentam utilizar conhecimentos prévios de relações semânticas para avaliar o sentimento das palavras e do texto [[9]](https://www.aclweb.org/anthology/J/J11/J11-2001.pdf). Nesse trabalho, escolhi utilizar o primeiro método por ser a base de muitos outros métodos mais complexos e utilizado com sucesso em aplicações práticas, como, por exemplo, [essa](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews) competição do Kaggle. Portanto, inicialmente pretendo me basear nos trabalhos [[4]](http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf) e [[5]](http://www.cs.cornell.edu/home/llee/papers/cutsent.pdf), o que me permite entender e aplicar um primeiro modelo funcional antes de entrar em métodos mais complexos. Em um futuro estudo, pretendo estudar outros métodos promissores, como métodos baseados em *Deep Learning* encontrados [aqui](https://nlp.stanford.edu/sentiment/), que permitem maior robustez ao analisar frases negativas e contextos mais complexos. Recentemente, métodos semelhantes criaram avanços significativos em análise de texto, como visto [neste](https://www.technologyreview.com/s/607828/an-algorithm-summarizes-lengthy-text-surprisingly-well/) artigo. Para maiores informações sobre diferentes métodos e suas características, os artigos [[10]](https://www.researchgate.net/publication/286302059_A_Benchmark_Comparison_of_State-of-the-Practice_Sentiment_Analysis_Methods) e [[3]](http://www.cs.cornell.edu/home/llee/omsa/omsa.pdf) apresentam um extenso e completo *benchmark* de vários métodos populares aplicados a problemas comuns de análise de sentimento. Outros artigos relacionados a análise de sentimento podem ser encontrados [aqui](http://www.cs.cornell.edu/people/pabo/movie-review-data/otherexperiments.html).

![Métodos usuais de análise de sentimento.](http://ars.els-cdn.com/content/image/1-s2.0-S2090447914000550-gr2.jpg)

- **Etapa 1**: Tendo definido um *framework* inicial, o próximo passo é analisar o *dataset* que disponho. Esse passo é crucial para poder entender possíveis problemas que posso encontrar ao realizar o processamento dos dados. Os principais problemas que procuro são possíveis palavras mal condicionadas (erros de digitação), forma da palavra (maiúscula e minúscula) ou possíveis problemas com delimitadores em geral (por exemplo, hífens, vírgulas, apóstrofes etc.) Outro problema comum que pretendo analisar são possíveis palavras que possuem contração no inglês, como, por exemplo, a contração negativa "isn't". Esse problema pode afetar diretamente algoritmos de análise de sentimentos baseados em *bag-of-words*. Outra etapa importante é a análise de palavras frequentes no texto em geral que não possuem correlação com o sentimento adotado no texto. Um exemplo disso são os artigos definidos e indefinidos. Para reduzir o efeito desse tipo de palavras, pode-se adotar métodos que reduzem o peso de palavras que não refletem novo conhecimento, como o cálculo do [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) de cada palavra. Esse método é particularmente interessante para métodos de *bag-of-words* que utilizam a frequência de cada palavra no texto para encontrar estatísticas úteis. Para métodos que se baseiam apenas na presença da palavra em um texto (por exemplo, [[4]](http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf) e [[5]](http://www.cs.cornell.edu/home/llee/papers/cutsent.pdf)), podemos apenas retirar as palavras irrelevantes ao contexto.

- **Etapa 2**: Após a análise e uma limpeza preliminar do texto, é possível iniciar o processamento dos dados. Como utilizarei o método de *bag-of-words*, o primeiro passo é criar um vetor das palavras que serão utilizadas no algoritmo de aprendizagem. No trabalho, pretendo comparar tanto o método de frequência relativa de cada palavra no texto, quanto o método da presença. Para ambos os métodos, a etapa preliminar é criar um dicionário com todas as palavras (com exceção das palavras retiradas no pré-processamento) de todos os exemplos de avaliação. Posteriormente, cada texto de avaliação é então codificado utilizando algum método em particular. Se for utilizado o método de frequência relativa, então é necessário codificar o texto em relação à frequência relativa de cada uma das palavras presente no dicionário criado anteriormente. Por exemplo, se um dicionário contem as palavras {eu, gosto, muito, bola, carro, bonito} e gostaríamos de criar um vetor codificado para o texto "eu gosto muito, muito [de]\* carro" (\* nesse caso considero que o "de" é retirado), então poderíamos criar um vetor com a quantidade de vezes que uma determinada palavra do dicionário é encontrada no texto. Portanto, o vetor ficaria [1 1 2 0 1 0]. Como dito anteriormente, essas quantidades podem ainda ser obtidas pelo cálculo do tf-idf, que aumentaria a robustez do modelo a palavras repetidas que não geram valor de sentimentos. A codificação do mesmo texto utilizando o método da presença seria semelhante ao anterior, mas agora só indicaríamos se a palavra do dicionário se encontra ou não no texto [[4]](http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf). Desse modo, o vetor se tornaria [1 1 1 0 1 0]. De acordo com Pang (2002) [[4]](http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf), espera-se que o método baseado em presença obtenha vantagem em relação ao método de frequência. Outra técnica que pretendo explorar é a utilização de *bigrams* para aumentar o entendimento de contexto do modelo. O *bigram* consiste em utilizar o conjunto de duas palavras adjacentes, que seria diferente do *unigram* que foi demonstrado até então ao exemplificar o *bag-of-words*. Por exemplo, enquanto a frase "não gosto" seria separada em {não} e {gosto} utilizando *unigrams*, a mesma frase seria separada em {não gosto} ao utilizar *bigram*. Pode-se ver claramente que há uma melhor análise de contexto ao avaliar o segundo conjunto, já que o *bigram* {não gosto} mostra claramente um efeito negativo, enquanto os *unigrams* {não} e {gosto} sozinhos não indicam claramente uma relação negativa ou positiva.

- **Etapa 3**: Agora tenho o sistema processado e pronto para realizar o treinamento. Como cada texto de avaliação do *dataset* possui uma indicação (*label*) prévia sobre o sentimento do autor, o método utilizado será da classe de aprendizagem de máquina [supervisionado](https://en.wikipedia.org/wiki/Supervised_learning). Além disso, o sistema deve classificar o texto entre positivo ou negativo, portanto, temos um problema de [classificação binária](https://en.wikipedia.org/wiki/Binary_classification). Um dos métodos mais comumente utilizado para esse fim é o *Support Vector Machine* ([SVM](https://en.wikipedia.org/wiki/Support_vector_machine)) [[3]](http://www.cs.cornell.edu/home/llee/omsa/omsa.pdf), que utiliza os dados de treinamento para criar um hiperplano que separa com maior distanciamento possível os pontos das classes diferentes que estão dispostos no espaço (veja figura abaixo). Diversos outros métodos poderiam ser utilizados para essa tarefa e o melhor caminho seria testar vários deles para resolver o problema, já que não existe um caminho único e universal para a resolução de problemas de aprendizado de máquina (a famosa frase "*no free lunch*" define!). Entretanto, o SVM foi aplicado em diversos problemas de análise de sentimento e sua eficácia foi comprovada [[4]](http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf). Mesmo assim, testarei o sistema utilizando o método de [regressão logística](https://web.stanford.edu/~jurafsky/slp3/7.pdf), [*Naive Bayes*](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) e, se houver tempo disponível, utilizarei um método de [redes neurais](https://en.wikipedia.org/wiki/Artificial_neural_network). Isso me permitirá tirar conclusões mais precisas sobre o sistema.

![Support Vector Machines.](http://docs.opencv.org/2.4/_images/optimal-hyperplane.png)

- **Etapa 4**: Após o treinamento, os métodos devem ser avaliados utilizando alguma métrica. O conjunto de dados será separado em três classes: treinamento (80%) e teste (20%). Os dados de treinamento serão utilizados para treinar o algoritmo de *machine learning* e o *dataset* de teste será usado para calcular a precisão do sistema com dados "desconhecidos" (isso é, não vistos em treinamento). O treinamento de cada método será validado a partir da [validação cruzada *k-fold*](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) utilizando os dados de treinamento (80% do *dataset*). Nesse trabalho, utilizarei *k*=10, ou seja, 9/10 dos dados de treinamento serão utilizados para treinamento e 1/10 será utilizado para validação do modelo. Esse processo será repetido *k* vezes, onde a escolha de que parte do *dataset* que será utilizada para treinamento e validação é aleatória. Após essa etapa, terei uma visão clara das vantagens e desvantagens de cada método. Ao final, testarei os modelos com os dados de teste (20% restantes do *dataset*) e utilizarei a acurácia como método final de avaliação.

- **Etapa 5**: Essa etapa é a última e se refere à conclusão do trabalho. Todas as análises e resultados serão apresentados e conclusões serão feitas a partir dos dados obtidos. Se houver resultados que fogem do esperado, será necessário repetir as etapas anteriores para avaliar se há algum erro de implementação do sistema. Se não houver, essa etapa tentará criar/testar hipóteses sobre as possíveis causas dos fenômenos. 

---
#### Conclusão

Esse texto tinha como objetivo apresentar as etapas que julgo necessárias para enfrentar o problema de classificação de sentimentos a partir de análise de texto. De fato, acredito que seja um *framework* comum para a análise e solução de qualquer problema novo que desejamos enfrentar. Essa metodologia nos permite criar um padrão de ações que nos facilita na tomada de decisões e evita a perda de tempo na resolução de problemas. Espero que tenha sido claro nas minhas explicações e, qualquer dúvida que tiver sobre o conteúdo, estou disponível para resolver a partir do meu e-mail.

---
### Referências:
[[1]](https://www.brandwatch.com/blog/understanding-sentiment-analysis/) Bannister, K. "Understanding Sentiment Analysis: What It Is & Why It’s Used", Brandwatch: https://www.brandwatch.com/blog/understanding-sentiment-analysis/, 2015;

[[2]](http://mashable.com/2010/04/19/sentiment-analysis/#KrPG2rUv45qh) Ogneva, M. "How Companies Can Use Sentiment Analysis to Improve Their Business", Mashable: http://mashable.com/2010/04/19/sentiment-analysis/#KrPG2rUv45qh, 2010;

[[3]](http://www.cs.cornell.edu/home/llee/omsa/omsa.pdf) Pang, B. e Lee, L. "Opinion mining and sentiment analysis", Foundations and Trends in Information Retrieval, 2008;

[[4]](http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf) Pang, B., Lee, L. e Vaithyanathan, S. "Thumbs up? Sentiment Classification using Machine Learning Techniques", Proceedings of EMNLP, 2002;

[[5]](http://www.cs.cornell.edu/home/llee/papers/cutsent.pdf) Pang, B. e Lee, L. "A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts", Proceedings of ACL, 2004;

[[6]](http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=9AF55F153515B1D5C27B68B9460B02AE?doi=10.1.1.129.4132&rep=rep1&type=pdf) Snyder, B. e Barzilay, R. "Multiple Aspect Ranking using the Good Grief Algorithm", Proceedings of NAACL HLT, 2007;

[[7]](http://www.sciencedirect.com/science/article/pii/S2090447914000550) Medhat, W., *et al*. "Sentiment analysis algorithms and applications: A survey", Ain Shams Engineering Journal, 2014

[[8]](https://arxiv.org/pdf/cs/0212032.pdf) Turney, P. "Thumbs Up or Thumbs Down? Semantic Orientation Applied to
Unsupervised Classification of Reviews", Proceedings of ACL, 2002;

[[9]](https://www.aclweb.org/anthology/J/J11/J11-2001.pdf) Taboada, M., *et al.* "Lexicon-Based Methods for
Sentiment Analysis", Journal Computational Linguistics, 2011;

[[10]](https://www.researchgate.net/publication/286302059_A_Benchmark_Comparison_of_State-of-the-Practice_Sentiment_Analysis_Methods) Gonçalves, P., et al. "A Benchmark Comparison of State-of-the-Practice Sentiment Analysis Methods", Preprint arXiv:1512.01818, 2015.

